In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.linear_model import LassoCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [3]:
train.set_index('Claim Identifier', inplace=True)
train.drop_duplicates(inplace=True)
train.drop('OIICS Nature of Injury Description', axis=1, inplace=True)

#train.drop(columns=['Birth Year', 'Age at Injury', 'Number of Dependents', 'WCIO Cause of Injury Code'], inplace=True)

In [4]:
train_num = train.select_dtypes(include=np.number).columns.tolist()
train_cat = train.select_dtypes(exclude=np.number).columns.tolist()

In [5]:
""" # Numerical columns: Impute with mean
num_imputer = SimpleImputer(strategy="mean")
train[train_num] = pd.DataFrame(
    num_imputer.fit_transform(train[train_num]),
    columns=train_num,
    index=train.index
)

# Categorical columns: Impute with most frequent (mode)
cat_imputer = SimpleImputer(strategy="most_frequent")
train[train_cat] = pd.DataFrame(
    cat_imputer.fit_transform(train[train_cat]),
    columns=train_cat,
    index=train.index
) """

' # Numerical columns: Impute with mean\nnum_imputer = SimpleImputer(strategy="mean")\ntrain[train_num] = pd.DataFrame(\n    num_imputer.fit_transform(train[train_num]),\n    columns=train_num,\n    index=train.index\n)\n\n# Categorical columns: Impute with most frequent (mode)\ncat_imputer = SimpleImputer(strategy="most_frequent")\ntrain[train_cat] = pd.DataFrame(\n    cat_imputer.fit_transform(train[train_cat]),\n    columns=train_cat,\n    index=train.index\n) '

In [6]:
X = train.drop(columns='Claim Injury Type')
y = train['Claim Injury Type']

In [7]:
X_num = X.select_dtypes(include=np.number).set_index(X.index)
X_cat = X.select_dtypes(exclude=np.number).set_index(X.index)

In [8]:
scaler = MinMaxScaler()
scaler.fit(X_num) #fit to training data
X_num_scaled = scaler.transform(X_num) # this will return an array
X_num_scaled = pd.DataFrame(X_num_scaled, columns = X_num.columns).set_index(X.index) # Convert the array to a pandas dataframe

In [9]:
X_cat = X_cat.astype(str)

enc1 = OrdinalEncoder() #encoder for features
enc2 = LabelEncoder() #encoder for labels
enc1.fit(X_cat)
X_cat_encoded = pd.DataFrame(enc1.transform(X_cat), columns = X_cat.columns).set_index(X.index)
y_encoded = enc2.fit_transform(y)

In [10]:
X_combined = pd.concat([X_num_scaled, X_cat_encoded], axis=1)
y_encoded_df = pd.DataFrame(y_encoded, columns=['Claim Injury Type'])

In [11]:
imputer = KNNImputer(n_neighbors=1).fit(X_combined)
filled_data = imputer.transform(X_combined)

In [12]:
""" imputer2 = KNNImputer(n_neighbors=2).fit(train)
filled_data2 = imputer2.transform(train) """

' imputer2 = KNNImputer(n_neighbors=2).fit(train)\nfilled_data2 = imputer2.transform(train) '

In [13]:
X_train, X_val, y_train, y_val = train_test_split(filled_data, y_encoded_df, train_size=0.75, stratify = y_encoded_df, random_state=5)

In [14]:
modelKNNT = KNeighborsClassifier(algorithm='kd_tree', n_neighbors=19).fit(X_train, y_train)

In [15]:
labels_train = modelKNNT.predict(X_train)
labels_val = modelKNNT.predict(X_val)

In [16]:
print(f1_score(y_train, labels_train, average='macro'))
print(f1_score(y_val, labels_val, average='macro'))

0.37220583986485295
0.35588027416137974
